In [1]:
import pandas as pd
import numpy as np 
data = pd.read_feather('database/GS.feather')

In [5]:
data

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,...,OMonth,OPeriod,Order Date Month,SYear,SMonth,SPeriod,Ship Date Month,Delivery,Price,Benefit
0,MX-2014-143658,2014-10-02,2014-10-06,Standard Class,SC-20575,Sonia Cooley,Consumer,Mexico City,Distrito Federal,Mexico,...,10,46,2014-10-01,2014,10,46,2014-10-01,4,4.36,1
1,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,10,22,2012-10-01,2012,10,22,2012-10-01,5,31.52,1
2,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,10,22,2012-10-01,2012,10,22,2012-10-01,5,96.64,1
3,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,10,22,2012-10-01,2012,10,22,2012-10-01,5,8.86,1
4,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,10,22,2012-10-01,2012,10,22,2012-10-01,5,35.80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,9,21,2012-09-01,2012,9,21,2012-09-01,2,11.07,1
51286,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,9,21,2012-09-01,2012,9,21,2012-09-01,2,30.72,1
51287,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,9,21,2012-09-01,2012,9,21,2012-09-01,2,20.13,1
51288,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,9,21,2012-09-01,2012,9,21,2012-09-01,2,32.61,1


In [10]:
def zscore(df, grupo, var, score):
    media = df[var].mean()
    dpadr = df[var].std()
    gr = df.groupby(grupo)[var]
    dc = gr.describe()
    dc = dc.reset_index()
    dc[score] = dc['mean'].apply(lambda x : (x - media) / dpadr)
    dcz = dc[[grupo, score]]
    df = df.merge(dcz, on=grupo, how='left')
    return df

In [1]:
## Classificacao zcore
dimensao = 'Customer ID'
medidas = ['Sales', 'Profit', 'Quantity']
grupo = data.groupby(dimensao)[medidas].mean().reset_index()
for medida in medidas: 
    grupo = zscore(grupo, dimensao, medida, 'z'+medida)
zmedidas = ['z'+med for med in medidas]
zgrupo = grupo[zmedidas]
grupo['score'] = zgrupo.sum(axis=1)
media_score = grupo['score'].mean()
dpadr_score = grupo['score'].std()
grupo['classe'] = grupo['score'].apply(lambda x : int((x - media_score) / dpadr_score) + 3)
grupo['rank'] = grupo['score'].rank(ascending=False)
grupo['lucro'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)
grupo

Dimensão ==> Segment


NameError: name 'data' is not defined

In [18]:
grupo.to_feather('database/classificacaoz_consumidor.feather')